# Data 처리

1. Read Data
2. 결측치 처리
3. 데이터 형 변환
4. 파생변수 생성

In [1]:
import pandas as pd
import numpy as np
# import pandas_profiling as pp
import gc

# Preprocessing & Feature Engineering
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler

# category encoding
from category_encoders.target_encoder import TargetEncoder
# from category_encoders.one_hoe import OneHotEncoder

# Utility
import os
import time
import random
import joblib
import re

pd.set_option('display.max_columns', 100)

## 0. 공통 변수 선언

In [2]:
features_cols = [] # 사용될 feature 리스트

## 1.Read Data & 처리

### 1.1 panel

In [3]:
panel = pd.read_csv('panel.csv')

C:\Users\user\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (7,27,29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


##### 결측치 없애기 전에 필요한 파생변수 생성 - panel 정보에 대한 응답율 피처 생성

In [4]:
# panel조사 분류별 응답율 피처 생성
SQ = ['SQ1', 'SQ2', 'SQ3', 'SQ4', 'SQ5', 'SQ6', 'SQ7', 'SQ8'] # 개인정보
A = ['A1'] # 신규구입/렌트 가전
B = ['B1', 'B2', 'B3', 'B4', 'B5'] # 통신
C = ['C1', 'C2', 'C3'] # 보험/금융
D = ['DQ1', 'DQ2', 'DQ3', 'DQ4', 'DQ5', 'DQ6', 'DQ7'] # 직업
F = ['F1', 'F2'] # 이용마트
H = ['H1'] # 음용주류
T = ['T1'] # 담배
X = ['X1', 'X2', 'X3', 'X4'] # 자동차

panel['SQ_R'] = 1 - panel[SQ].isnull().mean(axis=1)
panel['A_R'] = 1 - panel[A].isnull().mean(axis=1)
panel['B_R'] = 1 - panel[B].isnull().mean(axis=1)
panel['C_R'] = 1 - panel[C].isnull().mean(axis=1)
panel['D_R'] = 1 - panel[D].isnull().mean(axis=1)
panel['F_R'] = 1 - panel[F].isnull().mean(axis=1)
panel['H_R'] = 1 - panel[H].isnull().mean(axis=1)
panel['T_R'] = 1 - panel[T].isnull().mean(axis=1)
panel['X_R'] = 1 - panel[X].isnull().mean(axis=1)
# panel 조사 전체 응답율
panel['ALL_R'] = 1 - panel[SQ+A+B+C+D+F+H+T+X].isnull().mean(axis=1)

##### 결측치 제거, 교수님 로직 일부 수정

In [5]:
# 결측값 비율이 30% 이하인 column만 사용
features_cols = []
for f in panel.columns:
    if panel[f].isnull().sum()/panel.shape[0] <= 0.3:
        print(f, '\t', panel[f].nunique(), '\t', panel[f].isnull().sum()/panel.shape[0])
        features_cols.append(f)

userID 	 16563 	 0.0
BIRTH 	 76 	 6.037553583288052e-05
GENDER 	 2 	 6.037553583288052e-05
REGION 	 17 	 0.0006037553583288052
TYPE 	 3 	 0.0
SQ1 	 2 	 0.0013886373241562518
SQ2 	 83 	 0.001992392682485057
SQ3 	 53 	 0.09690273501177323
SQ4 	 46 	 0.0973253637626034
SQ5 	 277 	 0.15190484815552738
SQ6 	 25 	 0.15220672583469178
SQ7 	 1223 	 0.1849906417919459
SQ8 	 2083 	 0.18728491215359536
A1 	 2862 	 0.20956348487592827
B1 	 118 	 0.2172311779267041
B2 	 108 	 0.24035500815069732
B3 	 117 	 0.24403791583650306
B4 	 740 	 0.25381875264142967
B5 	 720 	 0.26323733623135903
SQ_R 	 9 	 0.0
A_R 	 2 	 0.0
B_R 	 6 	 0.0
C_R 	 4 	 0.0
D_R 	 8 	 0.0
F_R 	 3 	 0.0
H_R 	 2 	 0.0
T_R 	 2 	 0.0
X_R 	 5 	 0.0
ALL_R 	 33 	 0.0


In [6]:
# panel[panel['GENDER'].isnull()]
# TODO GENDER의 결측값을, SQ1로 채우는 것 고려 -> 1건
# panel[panel['REGION'].isnull()]
# TODO REGION의 결측값을, SQ3로 채우는 것 고려 -> 11건

In [7]:
# 불필요한 ID들은 제외. SQ1 ~ SQ3는 GENDER, BIRTH, REGION과 중복이어서 제외!
features_cols.remove('userID')
features_cols.remove('SQ1')
features_cols.remove('SQ2')
features_cols.remove('SQ3')

In [8]:
# # 거주 주택 형태: 1/2/3/4/5/6/97이 정상값
# panel.SQ4.value_counts()

In [9]:
def handle_profile(x):
    if x != x :
        return 0
    elif x in ['1', '2', '3', '4', '5', '6', '97']:
        return int(x)
    elif x in ['1,', '2,', '3,', '4,', '5,', '6,', '97,']:
        return int(x[0])
    else:
        return 98

panel.SQ4 = panel.SQ4.apply(handle_profile)

In [10]:
# # 최종학력: 1/2/3/4 정상값
# panel.SQ5.value_counts()#[:20]

In [11]:
def handle_profile(x):
    if x != x :
        return 0
    elif x in ['1', '2', '3', '4']:
        return int(x)
    elif x in ['1.0', '2.0', '3.0', '4.0']:
        return int(x[0])
    else:
        return 98

panel.SQ5 = panel.SQ5.apply(handle_profile)

In [12]:
# # 결혼여부: 1/2/97 정상값
# # TODO 3(25661건),4(31571건),5(9383건)의 값이 상당한 비중 차지 --> 전체 데이터가 잘못되었을 가능성..피처 제거 고려
# panel.SQ6.value_counts()

In [13]:
def handle_profile(x):
    if x != x :
        return 0
    elif x in ['1', '2']:
        return int(x)
    elif x in ['1,', '2,']:
        return int(x[0])
    else:
        return 98

panel.SQ6 = panel.SQ6.apply(handle_profile)

In [14]:
# # 자녀 정보: 1~16/99 정상값
# # 99도 의미 있는 값이므로, 이상값을 98로 치환
# panel.SQ7.value_counts()#[:40]

In [15]:
value1 = pd.Series(range(1,17)).apply(lambda x: str(x)).tolist()+['99']
value2 = pd.Series(np.arange(1.0,17.0)).apply(lambda x: str(x)).tolist()+['99.0']
def handle_profile(x):
    if x != x :
        return 0
    elif x in value1:
        return int(x)
    elif x in value2:
        return int(x[0])
    else:
        return 98

panel.SQ7 = panel.SQ7.apply(handle_profile)

In [16]:
# # 가족수: 1/2/3/4/5 정상값
# panel.SQ8.value_counts()#[:40]

In [17]:
def handle_profile(x):
    if x != x :
        return 0
    elif x in ['1', '2', '3', '4', '5']:
        return int(x)
    elif x in ['1.0', '2.0', '3.0', '4.0', '5.0']:
        return int(x[0])
    else:
        return 98

panel.SQ8 = panel.SQ8.apply(handle_profile)

In [18]:
# # 최근 구입/렌트 가전제품: 1~22/99 정상값
# # 99도 의미 있는 값이므로, 이상값을 98로 치환
# panel.A1.value_counts()#[:40]

In [19]:
values = pd.Series(range(1,23)).apply(lambda x: str(x)).tolist()+['99']
def handle_profile(x):
    if x != x :
        return 0
    elif x in value1:
        return int(x)
    else:
        return 98

panel.A1 = panel.A1.apply(handle_profile)

In [20]:
# # 휴대폰 통신망 종류: 1~8/99 정상값
# # 99도 의미 있는 값이므로, 이상값을 98로 치환
# panel.B1.value_counts()#[:35]

In [21]:
def handle_profile(x):    
    if x != x :
        return 0
    elif x in ['1', '1.', '1.']:
        return 2
    elif x in ['2', '3', '2.0', '3.0', '2,', '3,']:
        return 3
    elif x in ['4', '5', '6', '7', '4.0', '5.0', '6.0', '7.0', '4,', '5,', '6,', '7,']:
        return 4
    elif x in ['8', '8.0', '8,']:
        return 5
    else:
        return 99

panel.B1 = panel.B1.apply(handle_profile)

In [22]:
# # 가입한 이동통신사: 1~6/97 정상값 
# # 99도 정상으로 인식
# panel.B2.value_counts()#[:40]

In [23]:
def handle_profile(x):
    if x != x :
        return 0
    elif x in ['1', '2', '3', '4', '5', '6', '99']:
        return int(x)
    else:
        return 99

panel.B2 = panel.B2.apply(handle_profile)

In [24]:
# # 휴대폰 브랜드: 1~7/97/99
# panel.B3.value_counts()#[:40]

In [25]:
def handle_profile(x):    
    if x != x :
        return 0
    elif x in ['1', '2', '3', '4', '5', '6', '7']:
        return int(x)
    elif x in ['1,', '2,', '3,', '4,', '5,', '6,', '7,']:
        return int(x[0])
    elif x in ['99', '99,']:
        return 99
    else:
        return 4  # 기타 제조사

panel.B3 = panel.B3.apply(handle_profile)

In [26]:
# # 보유 스마트 기기: 1~7/97/99
# panel.B4.value_counts()#[:40]

In [27]:
def handle_profile(x):    
    if x != x :
        return 0
    elif x in ['1', '2', '3', '4', '5', '6', '7' , '99', '97']:
        return int(x)
    else:
        return 98

panel.B4 = panel.B4.apply(handle_profile)

In [28]:
# # 케이블TV: 1~10/97/99
# panel.B5.value_counts()#[:40]

In [29]:
def handle_profile(x):    
    if x != x :
        return 0
    elif x in ['1', '2', '3', '4' ,'5', '6', '7', '8', '9', '10']:
        return int(x)
    elif x in ['1,', '2,', '3,', '4,' ,'5,', '6,', '7,', '8,', '9,', '10,']:
        return int(x[0])
    elif x in ['99', '99,']:
        return 99
    else:
        return 4  # 기타 제조사

panel.B5 = panel.B5.apply(handle_profile)

In [30]:
panel

,userID,BIRTH,GENDER,REGION,TYPE,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7,SQ8,A1,B1,B2,B3,B4,B5,C1,C2,C3,F1,F2,H1,T1,X1,X2,X3,X4,DQ1,DQ2,DQ3,DQ4,DQ5,DQ6,DQ7,SQ_R,A_R,B_R,C_R,D_R,F_R,H_R,T_R,X_R,ALL_R
0,p00005,1979.0,2.0,2.0,D,2.0,1979.0,1,1,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.12500
1,p03884,1970.0,1.0,512.0,D,1.0,1970.0,8.0,1,3,2,4,3,98,4,4,2,98,6,"1,10,5,6,7,8",2,"1,9",1,1,"1,3",99,5,1.0,2,2,8,1,21,1,8.0,5.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.00000
2,p02205,1957.0,2.0,2.0,D,2.0,1957.0,1.0,2,3,1,9,2,99,3,3,1,2,5,"2,3",3,"1,10,2,3,4,5,6,9",3,1,99,99,2,1.0,"4,",1,5,12,2,7,1.0,6.0,3.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.00000
3,p12630,1997.0,1.0,512.0,B,1.0,1997.0,8.0,3,1,1,98,3,3,99,2,3,1,99,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.46875
4,p14431,1992.0,2.0,512.0,C,2.0,1992.0,8.0,1,3,1,3,98,3,3,5,4,2,4,4,2,"2,4,7,9",99,99,99,99,99,2.0,4,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.000000,0.0,1.0,1.0,1.0,1.0,0.78125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16558,p01702,1988.0,1.0,512.0,D,1.0,1988.0,8.0,1,3,2,9,3,98,5,1,1,98,4,"1,10,12,2,3,6,9",2,"2,4,6",7,2,"1,2,3",3,20,1.0,2,3.0,4,1,19,2,8.0,9.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.00000
16559,p05868,1993.0,1.0,512.0,B,1.0,1993.0,8.0,1,3,1,99,4,98,5,4,1,98,6,"11,9,1,3","5,","6,4","3,","2,",16,2,1,1.0,4,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.000000,0.0,1.0,1.0,1.0,1.0,0.78125
16560,p09312,1999.0,1.0,8.0,B,1.0,1999.0,7.0,1,1,98,98,5,3,99,99,99,99,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.43750
16561,p09181,1997.0,1.0,64.0,D,1.0,1997.0,3.0,1,2,1,99,4,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.25000


### 1.2 survey

In [31]:
survey = pd.read_csv('survey.csv')

In [32]:
for f in survey.columns:
    if survey[f].isnull().sum()/survey.shape[0] <= 0.3:
        print(f, '\t', survey[f].nunique(), 'b\t', survey[f].isnull().sum()/survey.shape[0])
        features_cols.append(f)

surveyID 	 5976 b	 0.0
TITLE 	 5955 b	 0.0
IR 	 75 b	 0.0
LOI 	 37 b	 0.0
CPI 	 83 b	 0.0


In [33]:
features_cols.remove('surveyID')

In [34]:
# # 대상지역 생성
survey['대상지역'] = np.where(survey['TITLE'].str.contains('해외'), '해외', '기타')
# # 대상자유형
survey['대상자유형'] = '기타'
survey['대상자유형'] = np.where(survey['TITLE'].str.contains('일반인'), '일반인', survey['대상자유형'])
survey['대상자유형'] = np.where(survey['TITLE'].str.contains('소비자'), '소비자', survey['대상자유형'])
features_cols.extend(['대상지역', '대상자유형'])

### 1.3 response_train/test

In [35]:
response_train = pd.read_csv('response_train.csv')
response_test = pd.read_csv('response_test.csv')

In [36]:
response_test.TIME.sort_values()

1183734    2021-06-01 01:00:04
1183892    2021-06-01 01:00:04
1183891    2021-06-01 01:00:04
1183854    2021-06-01 01:00:04
1183658    2021-06-01 01:00:04
                  ...         
758525     2021-10-18 00:37:29
758523     2021-10-18 00:37:29
759390     2021-10-18 00:37:29
759130     2021-10-18 00:37:29
758737     2021-10-18 00:37:29
Name: TIME, Length: 1355517, dtype: object

In [37]:
# 날짜/시간 관련 feature 생성
response_train.TIME = response_train.TIME.astype('datetime64')
response_train['MONTH'] = response_train.TIME.dt.month
response_train['DAY'] = response_train.TIME.dt.day
response_train['WEEKDAY'] = response_train.TIME.dt.day_of_week
response_train['HOUR'] = response_train.TIME.dt.hour

response_test.TIME = response_test.TIME.astype('datetime64')
response_test['MONTH'] = response_test.TIME.dt.month
response_test['DAY'] = response_test.TIME.dt.day
response_test['WEEKDAY'] = response_test.TIME.dt.day_of_week
response_test['HOUR'] = response_test.TIME.dt.hour

features_cols.extend(['MONTH', 'DAY', 'HOUR', 'WEEKDAY'])

### 1.4 train, test 생성(merge)

In [38]:
train = response_train.merge(panel).merge(survey)
test = response_test.merge(panel).merge(survey).sort_values(by='ID')

In [39]:
response_train.shape, train.shape, response_test.shape, test.shape

((4904549, 8), (4904549, 61), (1355517, 8), (1355517, 61))

### 1.9 Data 저장

In [41]:
with open('01.1.read_data.pkl', 'wb') as f:
    joblib.dump({'train' : train, 'test' : test, 'features_cols':features_cols}, f)

In [42]:
del train, test, features_cols, response_train, response_test
gc.collect()

64

## 2. 파생변수 생성

### 2.0 Load

In [43]:
with open('01.1.read_data.pkl', 'rb') as f:
    pkl_data = joblib.load(f)
locals().update(pkl_data)

In [44]:
train.shape, test.shape

((4904549, 61), (1355517, 61))

In [45]:
# response_train과 reponse_test 데이터의 특성이,
# response_test에 있는 모든 user는 reponse_train에 있다.
# 단, response_train에는 response_test보다 user가 많고
# panel에는 response_train보다 user가 더 있다.
list(set(test.userID.unique()) - set(train.userID.unique()))

[]

In [46]:
# train의 최소, 최대 일자
first_time = train.TIME.sort_values().iloc[1]
last_time = train.TIME.sort_values().iloc[-1]
first_time, last_time

(Timestamp('2020-06-01 02:30:11'), Timestamp('2021-05-31 23:02:19'))

In [47]:
# train의 데이터 기간을 test와 동일하게
test_first_time = test.TIME.sort_values().iloc[1]
test_last_time = test.TIME.sort_values().iloc[-1]
test_period = (test_last_time - test_first_time)
train_base_time = last_time - pd.DateOffset(days=test_period.days)
pre_train_base_time = train_base_time - pd.DateOffset(days=test_period.days) 

# 시간단위로 잘려서, 수작업으로 자른다. 
train_base_time = '2021-02-01'
pre_train_base_time = '2020-10-01'

all_train = train.copy() 
train = all_train[all_train.TIME >= train_base_time]
pre_train = all_train[(all_train.TIME < train_base_time) & (all_train.TIME >= pre_train_base_time)]

all_train.shape, train.shape, pre_train.shape

((4904549, 61), (1506055, 61), (2167232, 61))

In [48]:
# # 나눈 결과 --> 하루도 나눠져 있는게 있어, 일자별로 수정 해야 겠다.
# pre_train_base_time, train_base_time
# (Timestamp('2020-08-28 23:02:19'), Timestamp('2021-01-13 23:02:19'))

# test.TIME.sort_values().iloc[1], test.TIME.sort_values().iloc[-1]
# (Timestamp('2021-06-01 01:00:04'), Timestamp('2021-10-18 00:37:29'))

# train.TIME.sort_values().iloc[1], train.TIME.sort_values().iloc[-1]
# (Timestamp('2021-01-13 23:02:19'), Timestamp('2021-05-31 23:02:19'))

# pre_train.TIME.sort_values().iloc[1], pre_train.TIME.sort_values().iloc[-1]
# (Timestamp('2020-08-28 23:02:32'), Timestamp('2021-01-13 23:02:00'))


###### Title encoding 

In [49]:
# TITLE에 대한 한글 단어 빈도 합 encoding
word_counts = {}
def count_word(x): # 응답한 서베이 제목에서 한글 단어만 분리하고 빈도 계산
    if x['STATUS'] == 1:
        for w in re.sub(r'[^ ㄱ-ㅣ가-힣]', '', x['TITLE']).split():
            word_counts[w] = word_counts.get(w, 0) + 1
def score_word(x): # 빈도의 합으로 제목을 Encoding
    score = 0
    for w in re.sub(r'[^ ㄱ-ㅣ가-힣]', '', x['TITLE']).split():
        score += word_counts.get(w, 0)
    return score    
            
all_train.apply(count_word, axis=1)
# word_counts

0          None
1          None
2          None
3          None
4          None
           ... 
4904544    None
4904545    None
4904546    None
4904547    None
4904548    None
Length: 4904549, dtype: object

In [50]:
# TODO 수정해야 함. pre_train -> train -> test
all_train.TITLE = all_train.apply(score_word, axis=1)
pre_train.TITLE = pre_train.apply(score_word, axis=1)
train.TITLE = train.apply(score_word, axis=1)
test.TITLE = test.apply(score_word, axis=1)

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [51]:
all_train.shape, train.shape, pre_train.shape
# ((4904549, 61), (1749177, 61), (2511835, 61))

((4904549, 61), (1506055, 61), (2167232, 61))

###### 응답율 & 추이  & train의 현재로 부터 마지막 응답까지의 days
- 결측값을 없애기 위해, 전체 train 데이터로 값을 생성 후, pre_train과 train 처리

In [52]:
# 응답율
response = all_train.groupby('userID')['STATUS'].agg([
    ('RES_RATE', lambda x: np.mean(x))
]).reset_index()
all_train = all_train.merge(response, how='left')
pre_train = pre_train.merge(response, how='left')
train = train.merge(response, how='left')
test = pd.merge(test, response, how='left')

response = pre_train.groupby('userID')['STATUS'].agg([
    ('RES_RATE', lambda x: np.mean(x))
]).reset_index()
train.update(response)
test.update(response)

response = train.groupby('userID')['STATUS'].agg([
    ('RES_RATE', lambda x: np.mean(x))
]).reset_index()
test.update(response)

features_cols.extend(['RES_RATE'])

In [53]:
all_train.shape, train.shape, pre_train.shape

((4904549, 62), (1506055, 62), (2167232, 62))

In [54]:
# 최근 3개월 응답율 --> 5개월 씩 shift 하면 의미 없는 값인가?
response = all_train[all_train['TIME'] >= all_train.TIME.sort_values().iloc[-1] - pd.DateOffset(months=1)].groupby('userID')['STATUS'].agg([
    ('RES_RATE_3M', lambda x: np.mean(x))
]).reset_index()
all_train = all_train.merge(response, how='left')
pre_train = pre_train.merge(response, how='left')
train = train.merge(response, how='left')
test = pd.merge(test, response, how='left')

response = pre_train[pre_train['TIME'] >= pre_train.TIME.sort_values().iloc[-1] - pd.DateOffset(months=1)].groupby('userID')['STATUS'].agg([
    ('RES_RATE_3M', lambda x: np.mean(x))
]).reset_index()
train.update(response)
test.update(response)

response = train[train['TIME'] >= train.TIME.sort_values().iloc[-1] - pd.DateOffset(months=1)].groupby('userID')['STATUS'].agg([
    ('RES_RATE_3M', lambda x: np.mean(x))
]).reset_index()
test.update(response)

features_cols.extend(['RES_RATE_3M'])

In [55]:
all_train.shape, train.shape, pre_train.shape
# ((4904549, 61), (1749177, 61), (2511835, 61))

((4904549, 63), (1506055, 63), (2167232, 63))

In [56]:
# 타입별 응답율
response = all_train.groupby('TYPE')['STATUS'].agg([
    ('TYPE_RES_RATE', lambda x: np.mean(x))
]).reset_index()
all_train = all_train.merge(response, how='left', on='TYPE')
pre_train = pre_train.merge(response, how='left', on='TYPE')
train = train.merge(response, how='left', on='TYPE')
test = test.merge(response, how='left', on='TYPE')

features_cols.extend(['TYPE_RES_RATE'])

In [57]:
# 응답 추이 처리는 아래 null 처리 완료 후 실행

In [58]:
# 마지막 설문 후 지난 일자 : 클거나 null 값일 수록 
# response = train[train['STATUS']==1].groupby('userID')['TIME'].agg([
#     ('LAST_RES_DAYS', lambda x: (last_time-x.max()).days)
# ]).reset_index()
# train = train.merge(response, how='left')
# test = pd.merge(test, response, how='left')

# # 미리 결측값을 없앤다. 한번도 하지 않았을 경우 최대값
# train['LAST_RES_DAYS']= np.where(train['LAST_RES_DAYS'].isna(),
#                                  (last_time - first_time).days, train['LAST_RES_DAYS'])
# test['LAST_RES_DAYS']= np.where(test['LAST_RES_DAYS'].isna(),
#                                  (last_time - first_time).days, test['LAST_RES_DAYS'])

# features_cols.extend(['LAST_RES_DAYS'])

###### 패널별 응답 리워드, 난이도, 시간

In [59]:
# Train data 를 6개월 단위로 나누었을때 발생할 수 있는 문제를 고려하여 결측값에 대해 4.67개월 만큼의 비율만 반영하자.

In [60]:
# 패널의 총 리워드 포인트, 평균/최소/최대 리워드 포인트
response = all_train[all_train.STATUS==1].groupby('userID')['CPI'].agg([
    ('TOT_CPI', lambda x: np.sum(x)),
    ('AVG_CPI', lambda x: np.mean(x)),
    ('MIN_CPI', lambda x: np.min(x)),
    ('MAX_CPI', lambda x: np.max(x))
]).reset_index()
all_train = all_train.merge(response, how='left')
pre_train = pre_train.merge(response, how='left')
pre_train['TOT_CPI'] = pre_train['TOT_CPI'] * 1/3 #test_period.days/365
train = train.merge(response, how='left')
train['TOT_CPI'] = train['TOT_CPI'] * 1/3 #test_period.days/365
test = test.merge(response, how='left')
test['TOT_CPI'] = test['TOT_CPI'] * 1/3 #test_period.days/365

response = pre_train[pre_train.STATUS==1].groupby('userID')['CPI'].agg([
    ('TOT_CPI', lambda x: np.sum(x)),
    ('AVG_CPI', lambda x: np.mean(x)),
    ('MIN_CPI', lambda x: np.min(x)),
    ('MAX_CPI', lambda x: np.max(x))
]).reset_index()
train.update(response)
train['TOT_CPI'] = train['TOT_CPI'] * 2/3 #test_period.days/(365-test_period.days)
test.update(response)
test['TOT_CPI'] = test['TOT_CPI'] * 2/3 #test_period.days/(365-test_period.days)

response = train[train.STATUS==1].groupby('userID')['CPI'].agg([
    ('TOT_CPI', lambda x: np.sum(x)),
    ('AVG_CPI', lambda x: np.mean(x)),
    ('MIN_CPI', lambda x: np.min(x)),
    ('MAX_CPI', lambda x: np.max(x))
]).reset_index()
test.update(response)

features_cols.extend(['TOT_CPI', 'AVG_CPI', 'MIN_CPI', 'MAX_CPI'])

In [61]:
# 패널이 응한 설문의 평균/최소/최대 난이도
response = all_train[all_train.STATUS==1].groupby('userID')['IR'].agg([
    ('TOT_IR', lambda x: np.sum(x)),
    ('AVG_IR', lambda x: np.mean(x)),
    ('MIN_IR', lambda x: np.min(x)),
    ('MAX_IR', lambda x: np.max(x))
]).reset_index()
all_train = all_train.merge(response, how='left')
pre_train = pre_train.merge(response, how='left')
pre_train['TOT_IR'] = pre_train['TOT_IR'] * 1/3 #test_period.days/365
train = train.merge(response, how='left')
train['TOT_IR'] = train['TOT_IR'] * 1/3 #test_period.days/365
test = test.merge(response, how='left')
test['TOT_IR'] = test['TOT_IR'] * 1/3 #test_period.days/365

response = pre_train[pre_train.STATUS==1].groupby('userID')['IR'].agg([
    ('TOT_IR', lambda x: np.sum(x)),
    ('AVG_IR', lambda x: np.mean(x)),
    ('MIN_IR', lambda x: np.min(x)),
    ('MAX_IR', lambda x: np.max(x))
]).reset_index()
train.update(response)
train['TOT_IR'] = train['TOT_IR'] * 2/3 #test_period.days/(365-test_period.days)
test.update(response)
test['TOT_IR'] = test['TOT_IR'] * 2/3 #test_period.days/(365-test_period.days)

response = train[train.STATUS==1].groupby('userID')['IR'].agg([
    ('TOT_IR', lambda x: np.sum(x)),
    ('AVG_IR', lambda x: np.mean(x)),
    ('MIN_IR', lambda x: np.min(x)),
    ('MAX_IR', lambda x: np.max(x))
]).reset_index()
test.update(response)

features_cols.extend(['TOT_IR', 'AVG_IR', 'MIN_IR', 'MAX_IR'])

In [62]:
# 패널이 응한 설문의 평균/최소/최대 설문시간
response = all_train[all_train.STATUS==1].groupby('userID')['LOI'].agg([
    ('TOT_LOI', lambda x: np.sum(x)),
    ('AVG_LOI', lambda x: np.mean(x)),
    ('MIN_LOI', lambda x: np.min(x)),
    ('MAX_LOI', lambda x: np.max(x))
]).reset_index()
all_train = all_train.merge(response, how='left')
pre_train = pre_train.merge(response, how='left')
pre_train['TOT_LOI'] = pre_train['TOT_LOI'] * 1/3 #test_period.days/365
train = train.merge(response, how='left')
train['TOT_LOI'] = train['TOT_LOI'] * 1/3 #test_period.days/365
test = test.merge(response, how='left')
test['TOT_LOI'] = test['TOT_LOI'] * 1/3 #test_period.days/365

response = pre_train[pre_train.STATUS==1].groupby('userID')['LOI'].agg([
    ('TOT_LOI', lambda x: np.sum(x)),
    ('AVG_LOI', lambda x: np.mean(x)),
    ('MIN_LOI', lambda x: np.min(x)),
    ('MAX_LOI', lambda x: np.max(x))
]).reset_index()
train.update(response)
train['TOT_LOI'] = train['TOT_LOI'] * 2/3 #test_period.days/(365-test_period.days)
test.update(response)
test['TOT_LOI'] = test['TOT_LOI'] * 2/3 #test_period.days/(365-test_period.days)

response = train[train.STATUS==1].groupby('userID')['LOI'].agg([
    ('TOT_LOI', lambda x: np.sum(x)),
    ('AVG_LOI', lambda x: np.mean(x)),
    ('MIN_LOI', lambda x: np.min(x)),
    ('MAX_LOI', lambda x: np.max(x))
]).reset_index()
test.update(response)

features_cols.extend(['TOT_LOI', 'AVG_LOI', 'MIN_LOI', 'MAX_LOI'])

In [63]:
all_train.shape, train.shape, pre_train.shape
# ((4904549, 61), (1749177, 61), (2511835, 61))

((4904549, 76), (1506055, 76), (2167232, 76))

###### 요일 별 응답율

In [64]:
for w in np.arange(0,7):
    response = all_train[all_train['WEEKDAY']==w].groupby('userID')['STATUS'].agg([
        (f'WEEKDAY{w}_RES_RATE', lambda x: np.mean(x))
    ]).reset_index()
    all_train = all_train.merge(response, how='left')
    pre_train = pre_train.merge(response, how='left')
    train = train.merge(response, how='left')
    test = test.merge(response, how='left')
    
    response = pre_train[pre_train['WEEKDAY']==w].groupby('userID')['STATUS'].agg([
        (f'WEEKDAY{w}_RES_RATE', lambda x: np.mean(x))
    ]).reset_index()
    train.update(response)
    test.update(response)

    response = train[train['WEEKDAY']==w].groupby('userID')['STATUS'].agg([
        (f'WEEKDAY{w}_RES_RATE', lambda x: np.mean(x))
    ]).reset_index()
    test.update(response)
    
    features_cols.extend([f'WEEKDAY{w}_RES_RATE'])

In [65]:
all_train.shape, train.shape, pre_train.shape
# ((4904549, 61), (1749177, 61), (2511835, 61))

((4904549, 83), (1506055, 83), (2167232, 83))

###### 시간대 별 응답율 

In [66]:
HOURCLS_train_0 = (all_train['HOUR']>=7) & (all_train['HOUR']<13) # 오전
HOURCLS_train_1 = (all_train['HOUR']>=13) & (all_train['HOUR']<19) # 오후
HOURCLS_train_2 = (all_train['HOUR']>=19) & (all_train['HOUR']<24) # 저녁
HOURCLS_train_3 = (all_train['HOUR']>=0) & (all_train['HOUR']<7) # 야간

all_train['HOURCLS'] = 0
all_train['HOURCLS'] = np.where(HOURCLS_train_0, 0, all_train['HOURCLS'])
all_train['HOURCLS'] = np.where(HOURCLS_train_1, 1, all_train['HOURCLS'])
all_train['HOURCLS'] = np.where(HOURCLS_train_2, 2, all_train['HOURCLS'])
all_train['HOURCLS'] = np.where(HOURCLS_train_3, 3, all_train['HOURCLS'])

HOURCLS_train_0 = (pre_train['HOUR']>=7) & (pre_train['HOUR']<13) # 오전
HOURCLS_train_1 = (pre_train['HOUR']>=13) & (pre_train['HOUR']<19) # 오후
HOURCLS_train_2 = (pre_train['HOUR']>=19) & (pre_train['HOUR']<24) # 저녁
HOURCLS_train_3 = (pre_train['HOUR']>=0) & (pre_train['HOUR']<7) # 야간

pre_train['HOURCLS'] = 0
pre_train['HOURCLS'] = np.where(HOURCLS_train_0, 0, pre_train['HOURCLS'])
pre_train['HOURCLS'] = np.where(HOURCLS_train_1, 1, pre_train['HOURCLS'])
pre_train['HOURCLS'] = np.where(HOURCLS_train_2, 2, pre_train['HOURCLS'])
pre_train['HOURCLS'] = np.where(HOURCLS_train_3, 3, pre_train['HOURCLS'])

HOURCLS_train_0 = (train['HOUR']>=7) & (train['HOUR']<13) # 오전
HOURCLS_train_1 = (train['HOUR']>=13) & (train['HOUR']<19) # 오후
HOURCLS_train_2 = (train['HOUR']>=19) & (train['HOUR']<24) # 저녁
HOURCLS_train_3 = (train['HOUR']>=0) & (train['HOUR']<7) # 야간

train['HOURCLS'] = 0
train['HOURCLS'] = np.where(HOURCLS_train_0, 0, train['HOURCLS'])
train['HOURCLS'] = np.where(HOURCLS_train_1, 1, train['HOURCLS'])
train['HOURCLS'] = np.where(HOURCLS_train_2, 2, train['HOURCLS'])
train['HOURCLS'] = np.where(HOURCLS_train_3, 3, train['HOURCLS'])

HOURCLS_test_0 = (test['HOUR']>=7) & (test['HOUR']<13) # 오전
HOURCLS_test_1 = (test['HOUR']>=13) & (test['HOUR']<19) # 오후
HOURCLS_test_2 = (test['HOUR']>=19) & (test['HOUR']<24) # 저녁
HOURCLS_test_3 = (test['HOUR']>=0) & (test['HOUR']<7) # 야간

test['HOURCLS'] = 0
test['HOURCLS'] = np.where(HOURCLS_test_0, 0, test['HOURCLS'])
test['HOURCLS'] = np.where(HOURCLS_test_1, 1, test['HOURCLS'])
test['HOURCLS'] = np.where(HOURCLS_test_2, 2, test['HOURCLS'])
test['HOURCLS'] = np.where(HOURCLS_test_3, 3, test['HOURCLS'])

i=0
for cd in [0,1,2,3]:
    response = all_train[all_train['HOURCLS']==i].groupby('userID')['STATUS'].agg([
        (f'HOURCLS{i}_RES_RATE', lambda x: np.mean(x))
    ]).reset_index()
    all_train = all_train.merge(response, how='left')
    pre_train = pre_train.merge(response, how='left')
    train = train.merge(response, how='left')
    test = test.merge(response, how='left')
    
    response = pre_train[pre_train['HOURCLS']==i].groupby('userID')['STATUS'].agg([
        (f'HOURCLS{i}_RES_RATE', lambda x: np.mean(x))
    ]).reset_index()
    train.update(response)
    test.update(response)
    
    response = train[train['HOURCLS']==i].groupby('userID')['STATUS'].agg([
        (f'HOURCLS{i}_RES_RATE', lambda x: np.mean(x))
    ]).reset_index()
    test.update(response)

    features_cols.extend([f'HOURCLS{i}_RES_RATE'])
    i=i+1

In [67]:
# 타입별 응답율
response = all_train.groupby('HOURCLS')['STATUS'].agg([
    ('SURVEY_HOURCLS_RES_RATE', lambda x: np.mean(x))
]).reset_index()
all_train = all_train.merge(response, how='left', on='HOURCLS')
pre_train = pre_train.merge(response, how='left', on='HOURCLS')
train = train.merge(response, how='left', on='HOURCLS')
test = test.merge(response, how='left', on='HOURCLS')

features_cols.extend(['SURVEY_HOURCLS_RES_RATE'])

In [68]:
all_train.shape, train.shape, pre_train.shape
# ((4904549, 61), (1749177, 61), (2511835, 61))

((4904549, 89), (1506055, 89), (2167232, 89))

In [69]:
pre_train.head()

,userID,surveyID,TIME,STATUS,MONTH,DAY,WEEKDAY,HOUR,BIRTH,GENDER,REGION,TYPE,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7,SQ8,A1,B1,B2,B3,B4,B5,C1,C2,C3,F1,F2,H1,T1,X1,X2,X3,X4,DQ1,DQ2,DQ3,DQ4,DQ5,DQ6,DQ7,SQ_R,A_R,B_R,C_R,D_R,F_R,H_R,T_R,X_R,ALL_R,TITLE,IR,LOI,CATEGORIES,CPI,대상지역,대상자유형,RES_RATE,RES_RATE_3M,TYPE_RES_RATE,TOT_CPI,AVG_CPI,MIN_CPI,MAX_CPI,TOT_IR,AVG_IR,MIN_IR,MAX_IR,TOT_LOI,AVG_LOI,MIN_LOI,MAX_LOI,WEEKDAY0_RES_RATE,WEEKDAY1_RES_RATE,WEEKDAY2_RES_RATE,WEEKDAY3_RES_RATE,WEEKDAY4_RES_RATE,WEEKDAY5_RES_RATE,WEEKDAY6_RES_RATE,HOURCLS,HOURCLS0_RES_RATE,HOURCLS1_RES_RATE,HOURCLS2_RES_RATE,HOURCLS3_RES_RATE,SURVEY_HOURCLS_RES_RATE
0,p01216,s01352,2020-10-06 06:54:23,0,10,6,1,6,1994.0,1.0,2.0,D,1.0,1994.0,1.0,3,1,1,9,3,13,3,2,1,99,8,15,2,99,99,99,2,"99,,","99,",99.0,"99,",99,"97,아르바이트",99,99,99,99.0,4.0,99.0,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0000,4465987,30,10,"Pets,etc",550.0,기타,일반인,0.683183,0.911765,0.399483,94088.333333,620.362637,50.0,1650.0,10843.666667,71.496703,1.0,100.0,1979.000000,13.048352,1.0,30.0,0.659794,0.690265,0.743363,0.742268,0.641509,0.516129,0.730769,3,0.617512,0.612903,0.659574,0.754266,0.353289
1,p04916,s01352,2020-10-05 09:19:32,0,10,5,0,9,1987.0,2.0,2.0,B,2.0,1987.0,8,1,4,1,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1875,4465987,30,10,"Pets,etc",550.0,기타,일반인,0.084168,0.100000,0.279353,9853.333333,703.809524,50.0,2475.0,810.000000,57.857143,5.0,100.0,193.666667,13.833333,1.0,45.0,0.061538,0.095238,0.118421,0.115385,0.032258,0.114286,0.031250,0,0.082278,0.125000,0.063158,0.084211,0.308020
2,p03955,s01352,2020-10-05 09:19:25,1,10,5,0,9,1978.0,1.0,512.0,D,1.0,1985.0,1,1,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1250,4465987,30,10,"Pets,etc",550.0,기타,일반인,0.303226,0.259259,0.399483,30313.333333,644.964539,55.0,1650.0,3647.333333,77.602837,2.0,100.0,607.666667,12.929078,1.0,30.0,0.360656,0.266667,0.262295,0.357143,0.285714,0.287879,0.309091,0,0.260116,0.240741,0.287671,0.375758,0.308020
3,p02858,s01352,2020-10-05 09:19:28,0,10,5,0,9,1971.0,1.0,512.0,D,1.0,1971.0,8.0,1,3,2,8,3,13,4,4,1,98,6,"1,2",2,"97,지역화폐",2,1,"1,3",12,"5,",1.0,"2,",1,1,18,8,7,1.0,7.0,1.0,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0000,4465987,30,10,"Pets,etc",550.0,기타,일반인,0.544747,0.958333,0.399483,57790.000000,619.178571,50.0,1650.0,7423.000000,79.532143,2.0,100.0,1205.666667,12.917857,1.0,30.0,0.600000,0.415584,0.476744,0.708333,0.453333,0.616438,0.568627,0,0.603261,0.671233,0.358974,0.513966,0.308020
4,p01255,s01352,2020-10-05 09:19:11,0,10,5,0,9,1983.0,1.0,2.0,D,1.0,1983.0,1.0,3,3,1,9,3,99,4,1,1,98,4,"1,2,3",4,"1,2,6,9",1,1,99,99,99,99.0,"99,",99,99,99,99,99,99.0,5.0,11.0,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0000,4465987,30,10,"Pets,etc",550.0,기타,일반인,0.103239,0.083333,0.399483,11895.000000,699.705882,110.0,1705.0,931.333333,54.784314,2.0,100.0,232.333333,13.666667,2.0,31.0,0.125000,0.181818,0.100000,0.062500,0.102564,0.061538,0.095238,0,0.110465,0.030303,0.104478,0.121693,0.308020


In [70]:
train.head()

,userID,surveyID,TIME,STATUS,MONTH,DAY,WEEKDAY,HOUR,BIRTH,GENDER,REGION,TYPE,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7,SQ8,A1,B1,B2,B3,B4,B5,C1,C2,C3,F1,F2,H1,T1,X1,X2,X3,X4,DQ1,DQ2,DQ3,DQ4,DQ5,DQ6,DQ7,SQ_R,A_R,B_R,C_R,D_R,F_R,H_R,T_R,X_R,ALL_R,TITLE,IR,LOI,CATEGORIES,CPI,대상지역,대상자유형,RES_RATE,RES_RATE_3M,TYPE_RES_RATE,TOT_CPI,AVG_CPI,MIN_CPI,MAX_CPI,TOT_IR,AVG_IR,MIN_IR,MAX_IR,TOT_LOI,AVG_LOI,MIN_LOI,MAX_LOI,WEEKDAY0_RES_RATE,WEEKDAY1_RES_RATE,WEEKDAY2_RES_RATE,WEEKDAY3_RES_RATE,WEEKDAY4_RES_RATE,WEEKDAY5_RES_RATE,WEEKDAY6_RES_RATE,HOURCLS,HOURCLS0_RES_RATE,HOURCLS1_RES_RATE,HOURCLS2_RES_RATE,HOURCLS3_RES_RATE,SURVEY_HOURCLS_RES_RATE
0,p00000,s03253,2021-02-09 02:26:29,1,2,9,1,2,1994.0,1.0,2.0,D,1.0,1994.0,1.0,1,3,1,1,4,98,5,3,1,98,3,"10,13,14,5",8,"1,2,3,4,5,6,7,8,9","2,","3,","11,5,6","9,,","15,",1.0,"3,",1,4,16,99,"1,",3.0,2.0,1.0,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0000,2976252,100,14,NaN,770.0,해외,기타,0.737705,1.000000,0.399483,118630.000000,659.055556,110.0,1650.0,13721.333333,76.229630,2.0,100.0,2529.333333,14.051852,2.0,30.0,0.638889,0.741379,0.732143,0.642857,0.775000,0.843750,0.750000,3,0.763780,0.838235,0.469388,0.762295,0.353289
1,p00001,s03253,2021-02-02 00:31:43,1,2,2,1,0,1978.0,1.0,512.0,D,1.0,1985.0,1,1,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1250,2976252,100,14,NaN,770.0,해외,기타,0.627072,0.882353,0.399483,100783.333333,665.969163,110.0,2475.0,10862.666667,71.779736,2.0,100.0,2136.000000,14.114537,2.0,45.0,0.500000,0.568966,0.666667,0.675000,0.400000,0.742424,0.727273,3,0.552846,0.676923,0.471698,0.743802,0.353289
2,p00002,s03253,2021-02-02 00:31:43,1,2,2,1,0,1978.0,1.0,512.0,D,1.0,1985.0,8,1,3,1,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1875,2976252,100,14,NaN,770.0,해외,기타,0.212291,0.120968,0.399483,36540.000000,721.184211,220.0,1650.0,2492.666667,49.197368,10.0,100.0,696.666667,13.750000,4.0,30.0,0.185185,0.210526,0.328358,0.212766,0.205128,0.140625,0.175439,3,0.191176,0.217391,0.268293,0.214815,0.353289
3,p00003,s03253,2021-02-02 00:31:43,0,2,2,1,0,1979.0,1.0,8192.0,D,1.0,1979.0,13.0,1,4,2,1,3,98,4,1,1,98,6,"1,11,13,2,9",4,4,3,3,"1,3,7",99,2,1.0,"3,",2,4,17,1,2,8.0,11.0,1.0,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0000,2976252,100,14,NaN,770.0,해외,기타,0.135447,0.050847,0.399483,23370.000000,745.851064,50.0,1650.0,1292.666667,41.255319,5.0,100.0,433.333333,13.829787,1.0,30.0,0.047619,0.189655,0.178571,0.050000,0.290323,0.105263,0.113636,3,0.141791,0.020408,0.073171,0.195122,0.353289
4,p00004,s03253,2021-02-02 00:31:43,1,2,2,1,0,1982.0,1.0,2.0,D,1.0,1982.0,1.0,1,4,2,2,4,98,4,1,3,98,4,"1,10,11,12,13,14,15,2,3,4,5,6,7,8,9",8,"1,10,2,3,4,5,6,7,8,9","3,","1,","4,2,3,1,9","3,,","4,",1.0,"2,",2,1,16,7,3,8.0,10.0,1.0,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0000,2976252,100,14,NaN,770.0,해외,기타,0.626437,0.834783,0.399483,97806.666667,672.981651,50.0,1815.0,10568.666667,72.720183,5.0,100.0,2026.666667,13.944954,1.0,33.0,0.700000,0.518519,0.573333,0.574468,0.589744,0.796875,0.641026,3,0.581967,0.689655,0.593750,0.647059,0.353289


In [71]:
test.head()

,ID,userID,surveyID,TIME,MONTH,DAY,WEEKDAY,HOUR,BIRTH,GENDER,REGION,TYPE,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7,SQ8,A1,B1,B2,B3,B4,B5,C1,C2,C3,F1,F2,H1,T1,X1,X2,X3,X4,DQ1,DQ2,DQ3,DQ4,DQ5,DQ6,DQ7,SQ_R,A_R,B_R,C_R,D_R,F_R,H_R,T_R,X_R,ALL_R,TITLE,IR,LOI,CATEGORIES,CPI,대상지역,대상자유형,RES_RATE,RES_RATE_3M,TYPE_RES_RATE,TOT_CPI,AVG_CPI,MIN_CPI,MAX_CPI,TOT_IR,AVG_IR,MIN_IR,MAX_IR,TOT_LOI,AVG_LOI,MIN_LOI,MAX_LOI,WEEKDAY0_RES_RATE,WEEKDAY1_RES_RATE,WEEKDAY2_RES_RATE,WEEKDAY3_RES_RATE,WEEKDAY4_RES_RATE,WEEKDAY5_RES_RATE,WEEKDAY6_RES_RATE,HOURCLS,HOURCLS0_RES_RATE,HOURCLS1_RES_RATE,HOURCLS2_RES_RATE,HOURCLS3_RES_RATE,SURVEY_HOURCLS_RES_RATE
0,0,p00000,s04629,2021-06-16 12:43:33,6,16,2,12,1979.0,1.0,8.0,D,1.0,1979.0,7.0,1,3,1,9,1,1,4,5,3,1,7,1,2,99,5,99,"1,2","99,,","1,",1.0,"2,",1,8,22,13,1,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.00000,5775717,100,14,NaN,770.0,해외,일반인,0.982788,1.000000,0.399483,314020.0,549.947461,100.0,1705.0,54656.0,95.719790,2.0,100.0,7574.0,13.264448,1.0,31.0,1.000000,1.000000,0.944444,0.987952,0.988095,1.000000,0.962025,0,0.888889,0.988914,1.000000,0.982759,0.308020
1,1,p00001,s04629,2021-06-03 12:04:16,6,3,3,12,1998.0,2.0,8.0,D,2.0,1998.0,7.0,5,2,1,9,1,99,4,1,3,5,4,"5,7",9,"1,2,3,5,6,7,9",99,99,"1,3","99,,","99,",99.0,"99,",99,13,"97,편의점 아르바이트",12,"97,파트타이머",1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.00000,5775717,100,14,NaN,770.0,해외,일반인,0.910211,0.896739,0.399483,285500.0,552.224371,150.0,1705.0,48653.0,94.106383,2.0,100.0,6907.0,13.359768,1.0,31.0,0.882353,0.977778,0.926829,0.746835,0.986301,0.900000,0.949367,0,0.785714,0.911504,0.909091,0.984127,0.308020
2,2,p00002,s04629,2021-06-16 16:33:04,6,16,2,16,1981.0,1.0,2.0,D,1.0,1981.0,1,1,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.12500,5775717,100,14,NaN,770.0,해외,일반인,0.345382,0.434286,0.399483,99085.0,576.075581,110.0,1705.0,14843.0,86.296512,2.0,100.0,2310.0,13.430233,1.0,31.0,0.271605,0.424658,0.382353,0.493333,0.369863,0.210526,0.239437,1,0.174312,0.692308,0.628205,0.318792,0.364533
3,3,p00003,s04629,2021-06-16 16:32:34,6,16,2,16,1982.0,1.0,8.0,D,1.0,1982.0,7,1,3,2,99,2,99,4,1,2,99,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.43750,5775717,100,14,NaN,770.0,해외,일반인,0.068452,0.047619,0.399483,17655.0,767.608696,150.0,1650.0,779.0,33.869565,10.0,100.0,325.0,14.130435,1.0,30.0,0.085106,0.114754,0.066667,0.039216,0.068182,0.081633,0.000000,1,0.025381,0.058824,0.000000,0.160377,0.364533
4,4,p00004,s04629,2021-06-03 12:02:16,6,3,3,12,1999.0,1.0,32.0,B,1.0,1999.0,5,1,2,1,99,4,98,4,1,99,98,1,1,9,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.53125,5775717,100,14,NaN,770.0,해외,일반인,0.859296,0.820896,0.279353,98330.0,575.029240,150.0,1650.0,15604.0,91.251462,2.0,100.0,2275.0,13.304094,1.0,30.0,0.764706,0.846154,0.857143,0.969697,0.851852,0.885714,0.812500,0,0.822034,1.000000,0.900000,0.878049,0.308020


### 2.9 save pkl 

In [72]:
with open('01.2.make_feature_4M_shift_data.pkl', 'wb') as f:
    joblib.dump({'train' : train, 'test' : test, 'features_cols':features_cols}, f)

In [73]:
del all_train, pre_train, train, test, features_cols, response, word_counts
gc.collect()

42

## 3. Feature Engineering

### 3.0 Load 

In [74]:
with open('01.2.make_feature_4M_shift_data.pkl', 'rb') as f:
    pkl_data = joblib.load(f)
locals().update(pkl_data)

### 3.1 데이터 정리

In [75]:
# 학습용 데이터 분리를 고려하여 train을 TIME으로 sorting 한다.
train.sort_values(by='TIME', inplace=True)

In [76]:
y_train = train.STATUS

In [77]:
# train+test feature의 train, test 구분을 위한 index 저장
train_idx = train.shape[0]
train_idx

1506055

In [78]:
train.shape, test.shape

((1506055, 89), (1355517, 89))

In [79]:
# 데이터 전처리를 위한 train+test feature 생성
features = pd.concat([train[features_cols],test[features_cols]]).reset_index(drop=True)

In [80]:
del train, test
gc.collect()

126

### 3.2 Impute missing values

In [81]:
num_features = ['BIRTH','SQ_R', 'A_R', 'B_R', 'C_R', 'D_R', 'F_R', 'H_R', 'T_R', 'X_R', 'ALL_R',
                'TITLE', 'IR', 'LOI', 'CPI', 'MONTH', 'DAY', 'HOUR', 
                'RES_RATE', 'RES_RATE_3M','TYPE_RES_RATE', # 'LAST_RES_DAYS',
                'TOT_CPI', 'AVG_CPI', 'MIN_CPI', 'MAX_CPI', 'TOT_IR', 'AVG_IR', 'MIN_IR', 'MAX_IR',
                'TOT_LOI', 'AVG_LOI', 'MIN_LOI', 'MAX_LOI', 
                'WEEKDAY0_RES_RATE', 'WEEKDAY1_RES_RATE', 'WEEKDAY2_RES_RATE', 'WEEKDAY3_RES_RATE',
                'WEEKDAY4_RES_RATE', 'WEEKDAY5_RES_RATE', 'WEEKDAY6_RES_RATE',
                'HOURCLS0_RES_RATE', 'HOURCLS1_RES_RATE', 'HOURCLS2_RES_RATE', 'HOURCLS3_RES_RATE',
                'SURVEY_HOURCLS_RES_RATE']

In [82]:
cat_features = list(set(features.columns) - set(num_features))
cat_features

['B1',
 'SQ7',
 'GENDER',
 'TYPE',
 'SQ6',
 'REGION',
 'B3',
 'SQ4',
 '대상자유형',
 '대상지역',
 'B4',
 'B5',
 'A1',
 'SQ8',
 'B2',
 'WEEKDAY',
 'SQ5']

In [83]:
# 범주형 변수는 가장 많이 나온 값으로 채움
apply_most_frequent_features = cat_features 
features[apply_most_frequent_features + ['BIRTH']] = SimpleImputer(strategy='most_frequent').\
fit_transform(features[apply_most_frequent_features + ['BIRTH']])

In [84]:
# num feature 중 0으로 채울 대상
apply_zero_features = ['SQ_R', 'A_R', 'B_R', 'C_R', 'D_R', 'F_R', 'H_R', 'T_R', 'X_R', 'ALL_R',
                       'RES_RATE', 'RES_RATE_3M',
                       'TOT_CPI', 'AVG_CPI', 'MIN_CPI', 'MAX_CPI', 
                       'TOT_IR', 'AVG_IR', 'MIN_IR', 'MAX_IR',
                       'TOT_LOI', 'AVG_LOI', 'MIN_LOI', 'MAX_LOI',
                       'WEEKDAY0_RES_RATE', 'WEEKDAY1_RES_RATE', 'WEEKDAY2_RES_RATE', 'WEEKDAY3_RES_RATE',
                       'WEEKDAY4_RES_RATE', 'WEEKDAY5_RES_RATE', 'WEEKDAY6_RES_RATE',
                       'HOURCLS0_RES_RATE', 'HOURCLS1_RES_RATE', 'HOURCLS2_RES_RATE', 'HOURCLS3_RES_RATE'                       
                      ]
features[apply_zero_features] = SimpleImputer(strategy='constant', fill_value=0).\
fit_transform(features[apply_zero_features])

In [85]:
features

,BIRTH,GENDER,REGION,TYPE,SQ4,SQ5,SQ6,SQ7,SQ8,A1,B1,B2,B3,B4,B5,SQ_R,A_R,B_R,C_R,D_R,F_R,H_R,T_R,X_R,ALL_R,TITLE,IR,LOI,CPI,대상지역,대상자유형,MONTH,DAY,HOUR,WEEKDAY,RES_RATE,RES_RATE_3M,TYPE_RES_RATE,TOT_CPI,AVG_CPI,MIN_CPI,MAX_CPI,TOT_IR,AVG_IR,MIN_IR,MAX_IR,TOT_LOI,AVG_LOI,MIN_LOI,MAX_LOI,WEEKDAY0_RES_RATE,WEEKDAY1_RES_RATE,WEEKDAY2_RES_RATE,WEEKDAY3_RES_RATE,WEEKDAY4_RES_RATE,WEEKDAY5_RES_RATE,WEEKDAY6_RES_RATE,HOURCLS0_RES_RATE,HOURCLS1_RES_RATE,HOURCLS2_RES_RATE,HOURCLS3_RES_RATE,SURVEY_HOURCLS_RES_RATE
0,2004.0,1.0,2.0,B,1,1,98,98,3,1,2,99,2,98,2,1.0,1.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.43750,128,15,22,1210.0,기타,기타,2,1,1,0,0.000000,0.000000,0.279353,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.353289
1,1995.0,2.0,2.0,B,1,1,1,98,1,1,99,99,99,4,3,1.0,1.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.43750,128,15,22,1210.0,기타,기타,2,1,1,0,0.009415,0.008403,0.279353,2777.777778,520.833333,45.0,1650.0,276.222222,51.791667,10.0,100.0,54.444444,10.208333,1.0,30.0,0.005450,0.008130,0.008753,0.032609,0.007229,0.000000,0.000000,0.015267,0.002401,0.000000,0.034632,0.353289
2,1994.0,2.0,2.0,B,1,98,98,2,98,98,4,0,0,0,0,1.0,1.0,0.2,0.0,0.000000,0.0,0.0,0.0,0.00,0.31250,128,15,22,1210.0,기타,기타,2,1,1,0,0.009030,0.016736,0.279353,4288.888889,839.130435,220.0,1650.0,153.111111,29.956522,0.0,100.0,78.666667,15.391304,4.0,30.0,0.011111,0.008152,0.006550,0.013477,0.017157,0.003236,0.000000,0.024490,0.000000,0.000000,0.036093,0.353289
3,2002.0,1.0,2.0,B,2,1,98,98,1,1,2,99,2,98,7,1.0,1.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.43750,128,15,22,1210.0,기타,기타,2,1,1,0,0.000000,0.000000,0.279353,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.353289
4,1986.0,2.0,512.0,D,3,0,0,0,0,0,0,0,0,0,0,0.5,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.12500,128,15,22,1210.0,기타,기타,2,1,1,0,0.018231,0.021097,0.399483,6287.777778,602.021277,50.0,1650.0,530.666667,50.808511,10.0,100.0,123.555556,11.829787,1.0,30.0,0.033592,0.032967,0.016842,0.011331,0.019370,0.000000,0.007092,0.072464,0.002395,0.000000,0.048936,0.353289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2861567,1995.0,1.0,2.0,B,4,3,1,99,4,98,4,3,3,98,8,1.0,1.0,1.0,1.0,0.285714,1.0,1.0,1.0,1.00,0.84375,5775717,100,16,880.0,해외,일반인,9,12,9,6,0.210000,0.242424,0.279353,3615.555556,774.761905,100.0,1650.0,168.666667,36.142857,3.0,90.0,66.000000,14.142857,3.0,30.0,0.190476,0.250000,0.083333,0.266667,0.363636,0.250000,0.117647,0.178571,0.000000,0.083333,0.312500,0.308020
2861568,1981.0,2.0,512.0,C,1,3,2,3,98,1,3,99,2,98,4,1.0,1.0,1.0,1.0,0.000000,1.0,1.0,1.0,0.75,0.75000,5775717,100,16,880.0,해외,일반인,9,5,7,6,0.065789,0.038462,0.090127,575.555556,518.000000,480.0,600.0,97.777778,88.000000,40.0,100.0,13.555556,12.200000,10.0,15.0,0.111111,0.000000,0.181818,0.000000,0.076923,0.000000,0.111111,0.000000,0.052632,0.076923,0.136364,0.308020
2861569,1984.0,2.0,2.0,C,1,4,1,3,98,4,3,1,3,2,4,1.0,1.0,1.0,1.0,0.000000,1.0,1.0,1.0,0.75,0.75000,5775717,100,16,880.0,해외,일반인,9,5,2,6,0.100000,0.193548,0.090127,646.666667,485.000000,150.0,600.0,131.111111,98.333333,90.0,100.0,16.000000,12.000000,3.0,15.0,0.125000,0.000000,0.125000,0.000000,0.100000,0.125000,0.153846,0.150000,0.200000,0.000000,0.043478,0.353289
2861570,1987.0,1.0,256.0,C,1,3,1,3,98,5,3,99,3,2,4,1.0,1.0,1.0,1.0,0.000000,1.0,1.0,1.0,1.00,0.78125,5775717,100,16,880.0,해외,일반인,10,5,0,1,0.122222,0.102564,0.090127,1435.555556,587.272727,480.0,825.0,220.000000,90.000000,40.0,100.0,33.555556,13.727273,12.0,15.0,0.214286,0.363636,0.000000,0.000000,0.230769,0.000000,0.083333,0.076923,0.111111,0.111111,0.162162,0.353289


###### 응답율 추이 파생변수 추가 

In [86]:
# 응답율 추이 변수 추가
features['RES_RATE_TREND'] = (features['RES_RATE_3M'] - features['RES_RATE'])# / features['RES_RATE']

In [87]:
num_features.extend(['RES_RATE_TREND'])

In [88]:
#0으로 채우는게 맞을까...유지하는 것이니...맞다고 볼 수 있을듯
features[['RES_RATE_TREND']] = SimpleImputer(strategy='constant', fill_value=0).\
fit_transform(features[['RES_RATE_TREND']])

In [89]:
# 응답율을 제외한 나머지 숫자형 변수는 mean 값 채우기
apply_mean_features = list(set(num_features) - set(apply_zero_features))
features[apply_mean_features] = SimpleImputer(strategy='mean').\
fit_transform(features[apply_mean_features])

In [90]:
features['GENDER'] = features['GENDER'].astype(int)
features['REGION'] = features['REGION'].astype(int)

### 3.2-1 Transform features (수치형 변수에 대해서만 진행)

In [91]:
# features[num_features] = PowerTransformer(standardize=True).fit_transform(features[num_features])

### 3.4 Transform features (feature Scaling) 

In [92]:
# DNN 모델링에서는 StandardScaler을 주로 사용
sacling_features_col = features.columns.tolist()
scaler = StandardScaler()
features[num_features] = scaler.fit_transform(features[num_features])

### 3.3 범주형 변수 인코딩 

In [94]:
te_features = features.copy()
ohe_feature = features.copy()

In [95]:
encoder = TargetEncoder(cols=cat_features)
encoder.fit(te_features[cat_features].iloc[:train_idx, :], y_train)
te_features[cat_features] = encoder.transform(te_features[cat_features])

C:\Users\user\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [96]:
# ohe_feature =  pd.get_dummies(ohe_feature, columns=cat_features)

### 3.9. save pkl

In [97]:
# train과 test로 재 분할
# X_train = features.iloc[:train_idx, :]

# X_test = features.iloc[train_idx:, :]
# X_test = X_test.reset_index(drop=True)
# del features
# gc.collect()

In [98]:
# te_features.iloc[:train_idx, :].shape, ohe_feature.iloc[:train_idx, :].shape, X_test.shape

##### feature selection

In [ ]:
# Manually or SHAP 
features_selected = [
'RES_RATE_3M',
'RES_RATE', 'HOURCLS3_RES_RATE',
'TITLE',
'LOI', 'CPI', 'TOT_IR',
'TYPE_RES_RATE', 'SURVEY_HOURCLS_RES_RATE',
'MAX_CPI', 'AVG_LOI',
'HOURCLS1_RES_RATE',
'HOUR',
'AVG_CPI', 'IR', 'AVG_IR',
'HOURCLS0_RES_RATE', 'RES_RATE_TREND',
]

X_train, X_test = X_train[features_selected], X_test[features_selected]

In [99]:
#with open('01.3.feature_target_shift_4M_home.pkl', 'wb') as f:
#    joblib.dump({'X_train' : te_features.iloc[:train_idx, :], 
#                 'X_test' : te_features.iloc[train_idx:, :].reset_index(drop=True), 'y_train':y_train}, f)

In [100]:
with open('01.3.feature_ohe_shift_4M_home.pkl', 'wb') as f:
    joblib.dump({'X_train' : ohe_feature.iloc[:train_idx, :], 
                 'X_test' : ohe_feature.iloc[train_idx:, :].reset_index(drop=True), 'y_train':y_train, 'cat_features':cat_features}, f)

In [101]:
del features, te_features, ohe_feature, features_cols, y_train
gc.collect()

126